In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
%ls

BankNote.ipynb*               TF.ipynb
MNIST.ipynb                   TensorFlow - Belajar.ipynb
MNIST_data/                   Untitled.ipynb
TF Estimator - Belajar.ipynb  bank_note_data.csv*
TF Estimator.ipynb            iris.csv*


In [3]:
df = pd.read_csv('iris.csv')

In [4]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [5]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [6]:
df.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "target"]

In [7]:
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target'], dtype='object')

In [8]:
df['target'] = df['target'].apply(int)

In [ ]:
df.head()

In [10]:
X = df.drop('target', axis=1)
y = df['target']

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [15]:
import tensorflow as tf

In [16]:
feature_cols = []
for col in X.columns:
    feature_cols.append(tf.feature_column.numeric_column(col))

In [53]:
feature_cols

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [18]:
input_function = tf.estimator.inputs.pandas_input_fn
    (x=X_train,y=y_train,batch_size=10,num_epochs=5,shuffle=True)

In [63]:
classifier = tf.estimator.DNNClassifier(hidden_units=[10,20,20], n_classes=3, feature_columns=feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/gx/l4cmby291772nllh4y00mc600000gn/T/tmpaillvmx_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12a338240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [64]:
classifier.train(input_fn=input_function,steps=50)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/gx/l4cmby291772nllh4y00mc600000gn/T/tmpaillvmx_/model.ckpt.
INFO:tensorflow:loss = 14.216412, step = 1
INFO:tensorflow:Saving checkpoints for 50 into /var/folders/gx/l4cmby291772nllh4y00mc600000gn/T/tmpaillvmx_/model.ckpt.
INFO:tensorflow:Loss for final step: 2.96142.


In [65]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=len(X_test), shuffle=False)

In [66]:
prediction = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/gx/l4cmby291772nllh4y00mc600000gn/T/tmpaillvmx_/model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [67]:
prediction[5]

{'logits': array([-4.259362 ,  0.5995781,  1.8951608], dtype=float32),
 'probabilities': array([0.00166464, 0.21455161, 0.7837837 ], dtype=float32),
 'class_ids': array([2]),
 'classes': array([b'2'], dtype=object)}

In [68]:
final_preds = []
for pred in prediction:
    final_preds.append(pred['class_ids'][0])

In [69]:
from sklearn.metrics import classification_report, confusion_matrix

In [70]:
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      0.85      0.92        13
           2       0.88      1.00      0.94        15

   micro avg       0.96      0.96      0.96        45
   macro avg       0.96      0.95      0.95        45
weighted avg       0.96      0.96      0.96        45

